# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [334]:
            # Passo a Passo

# Importar as bibliotecas

# Criar um navegador

# Importar/visualizar/tratar a Base de dados

# Para cada produto
    # procurar produto no Google Shopping
        # Verificar se o produto ta na faixa de preço indicada
    # procurar produto no Buscapé
        # Verificar se o produto ta na faixa de preço indicada

# Salvar as ofertas em uma tabela

# Exportar um arqivo com as informações da Tabela

# Enviar por e-mail o resultado

In [335]:
# Importar as bibliotecas
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# LEMBRETE : SELENIUM DEVE ESTAR INSTALADO NO SEU COMPUTADOR!

In [336]:
# Criar um navegador
navegador = webdriver.Chrome()

In [337]:
# Importar/visualizar/tratar a Base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


#### Definindo função de busca

In [338]:
# Procurar produto no Google Shopping (transformando em uma função)

def busca_google_shopping(navegador, produto, termos_banido, preco_min, preco_max):
    '''
    Função que entra no google, pesquisa produto, removendo termos banidos e retorna o nome, valor e link do produto 
    conforme filtro de preço minimo e máximo
    '''

    # Entrar no Google
    navegador.get('https://www.google.com/')

    # tratar valores da tavela para pesquisa

    produto = produto.lower() # garantir que não haja diferenciação de maisuculas e minusculas
    lista_termos_produto = produto.split(' ')

    termos_banido = termos_banido.lower() # garantir que não haja diferenciação de maisuculas e minusculas
    lista_termos_banido = termos_banido.split(' ')

    # Pesquisar
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    # Clicar em Shopping
    elementos = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for elemento in elementos:
        if 'Shopping' in elemento.text:
            elemento.click()
            break
        else:
            navegador.find_element(By.CLASS_NAME, 'GOE98c').click()
            elementos = navegador.find_elements(By.CLASS_NAME, 'tnhqA')
            for elemento in elementos:
                if 'Shopping' in elemento.text:
                    print(elemento)
                    elemento.click()
                    break

    # Pegar informações de descrição, preço e link e criar uma lista
    cards = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    lista_ofertas_google = []   # Lista para conter as informações dos produtos encontrados após filtragem
    
    for card in cards:
        nome = card.find_element(By.CLASS_NAME, 'Xjkr3b').text # Busca Todos os Nomes de Produtos
        nome = nome.lower() # garantir que não haja diferenciação de maisuculas e minusculas

        tem_termos_banidos = False
        for palavra in lista_termos_banido:   #filtra os itens evitando termos banidos
            if palavra in nome:
                tem_termos_banidos = True

        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:   #filtra os itens garantindo que tenha todos os termos da pesquisa
            if palavra not in nome:
                tem_todos_termos_produto = False

        # Se tem todos os termos do produto e não tem termos banidos, queremos verificar o preço.
        if not tem_termos_banidos and tem_todos_termos_produto:
            try:
                preco = card.find_element(By.CLASS_NAME, 'a8Pemb').text # vem em formato de texto e com R$, deve ser tratado
                preco = preco.replace('R$', '').replace(' ','').replace('.','').replace(',','.')
                preco = float(preco)
                preco_max = float(preco_max)
                preco_min = float(preco_min)
        # verificando se o preço está dentro do preço mínimo e máximo
                if preco_min <= preco <= preco_max:
                    elemento_link = card.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas_google.append((nome, preco, link))
            except:
                continue
    return lista_ofertas_google


def busca_buscape(navegador, produto, termos_banido, preco_min, preco_max):
    '''
    Função que entra no buscpé, pesquisa produto, removendo termos banidos e retorna o nome, valor e link do produto 
    conforme filtro de preço minimo e máximo
    '''

    # Entrar no BuscaPé
    navegador.get('https://www.buscape.com.br/')

    # tratar valores da tavela para pesquisa

    produto = produto.lower() # garantir que não haja diferenciação de maisuculas e minusculas
    lista_termos_produto = produto.split(' ')

    termos_banido = termos_banido.lower() # garantir que não haja diferenciação de maisuculas e minusculas
    lista_termos_banido = termos_banido.split(' ')

    # Pesquisar
    search = 0
    while search == 0:
        try:
            search = navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_textBox__eLv3V').send_keys(produto, Keys.ENTER)
            break
        except:
            time.sleep(1)
            continue    
    
    time.sleep(5)
    
    # Pegar informações de descrição, preço e link e criar uma lista
    
    #navegador.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')
    
    cards = 0
    while cards == 0:
        try:
            cards = navegador.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')
            break
        except:
            time.sleep(1)
            continue
    
    lista_ofertas_buscape = []   # Lista para conter as informações dos produtos encontrados após filtragem
    
    for card in cards:
        nome = card.get_attribute('title') # Busca Todos os Nomes de Produtos
        nome = nome.lower() # garantir que não haja diferenciação de maisuculas e minusculas

        tem_termos_banidos = False
        for palavra in lista_termos_banido:   #filtra os itens evitando termos banidos
            if palavra in nome:
                tem_termos_banidos = True

        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:   #filtra os itens garantindo que tenha todos os termos da pesquisa
            if palavra not in nome:
                tem_todos_termos_produto = False

        # Se tem todos os termos do produto e não tem termos banidos, queremos verificar o preço.
        if not tem_termos_banidos and tem_todos_termos_produto:
            
            preco = card.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text # vem em formato de texto e com R$, deve ser tratado
            preco = preco.replace('R$', '').replace(' ','').replace('.','').replace(',','.')
            preco = float(preco)

        # verificando se o preço está dentro do preço mínimo e máximo

            preco_max = float(preco_max)
            preco_min = float(preco_min)
            if preco_min <= preco <= preco_max:  
                link = card.get_attribute('href')
                lista_ofertas_buscape.append((nome, preco, link))
    return lista_ofertas_buscape

In [339]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banido = tabela_produtos.loc[linha, 'Termos banidos']
    preco_min = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_max = tabela_produtos.loc[linha, 'Preço máximo']

    lista_ofertas_google = busca_google_shopping(navegador, produto, termos_banido, preco_min, preco_max)
    if lista_ofertas_google:
        tabela_google = pd.DataFrame(lista_ofertas_google, columns=['nome','preco','link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google)
    else:
        tabela_google = None
        
    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banido, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['nome','preco','link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
    
display(tabela_ofertas)

C:\Users\User\AppData\Local\Temp\ipykernel_8748\2032741985.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google)
C:\Users\User\AppData\Local\Temp\ipykernel_8748\2032741985.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_buscape)
C:\Users\User\AppData\Local\Temp\ipykernel_8748\2032741985.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google)
C:\Users\User\AppData\Local\Temp\ipykernel_8748\2032741985.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.a

,nome,preco,link
0,iphone 12 (64gb) - grade a - excelente (azul),3299.00,https://www.google.com/url?url=https://jobsimp...
1,"iphone 11 branco, com tela de 6,1, 4g, 64 gb e...",3367.99,https://www.google.com/url?url=https://www.sho...
0,smartphone apple iphone se 2 vermelho 64gb 12....,3002.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de video msi geforce rtx 3060 ti gaming ...,4180.90,https://www.google.com/url?url=https://www.net...
1,placa de vídeo msi geforce rtx 3060 ti ventus ...,4150.00,https://www.google.com/url?url=https://www.gue...
2,placa de video colorful igame geforce rtx 3060...,4200.00,https://www.google.com/url?url=https://www.cnt...
3,placa de vídeo asus nvidia geforce dual rtx306...,4116.92,https://www.google.com/url?url=https://www.cas...
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,4257.25,https://www.buscape.com.br/placa-de-video/plac...
1,placa de video nvidia geforce rtx 3060 ti 8 gb...,4376.62,https://www.buscape.com.br/placa-de-video/plac...


### Gerando arquivo de Excel da Tabela de Ofertas


In [340]:
# Exportar a Tabela de Ofertas pro Excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

### Enviando e-mail das Ofertas


In [341]:
# Só enviar e-mail, se Houver alguma oferta

#verificando
if len(tabela_ofertas.index) > 0:
    # caso exista, enviar e-mail.

    import win32com.client as win32
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'islam.igor93@gmail.com'
    mail.Subject = 'Ofertas Encontradas'
    mail.HTMLBody = f'''
    <p>Prezados,</p>
    <p>Encontramos ofertas na faixa de preço desejada</p>
    {tabela_ofertas.to_html()}
    <p>Qualquer dúvida, estou a disposição</p>
    '''
    mail.Send()
    
navegador.quit()